In [31]:
import plotly.express as px
import streamlit as st
import plotly.io as pio
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime


from mainnet_launch.constants import *

from mainnet_launch.database.schema.full import Destinations, AutopoolDestinations, Tokens, SwapQuote, AssetExposure
from mainnet_launch.database.schema.postgres_operations import (
    get_full_table_as_df,
    get_highest_value_in_field_where,
    _exec_sql_and_cache,
)
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"


df = get_full_table_as_df(SwapQuote)
df["effective_price"] = df["scaled_amount_out"] / df["scaled_amount_in"]
tokens_df = get_full_table_as_df(Tokens)
token_address_to_symbol = dict(zip(tokens_df["token_address"], tokens_df["symbol"]))
df["buy_token_symbol"] = df["buy_token_address"].map(token_address_to_symbol)
df["sell_token_symbol"] = df["sell_token_address"].map(token_address_to_symbol)

df["day"] = df["datetime_received"].dt.date
print(df["day"].value_counts())
df = df[df["datetime_received"].dt.date == datetime.date(2025, 8, 13)].reset_index(drop=True)
df

day
2025-08-13    10500
2025-08-12     4510
Name: count, dtype: int64


,chain_id,base_asset,api_name,sell_token_address,buy_token_address,scaled_amount_in,scaled_amount_out,pools_blacklist,percent_exclude_threshold,aggregator_name,datetime_received,quote_batch,id,effective_price,buy_token_symbol,sell_token_symbol,day
0,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,odos,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,0x865377367054516e17014CcdED1e7d814EDC9ce4,800000.0,848296.913572,None,99.0,Odos,2025-08-13 04:23:10.753600+00:00,7,a334b065-b0f0-492f-b564-0c275e5b92c6,1.060371,DOLA,scrvUSD,2025-08-13
1,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,odos,0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,200.0,210.323659,None,99.0,Odos,2025-08-13 04:25:25.729926+00:00,7,326c5269-5843-408e-8ed1-7a2a30c570be,1.051618,WETH,osETH,2025-08-13
2,8453,0x4200000000000000000000000000000000000006,odos,0x04C0599Ae5A44757c0af6F9eC3b93da8976c150A,0x4200000000000000000000000000000000000006,375.0,341.885033,None,99.0,Odos,2025-08-13 04:24:40.808307+00:00,7,854caa5a-c56e-459a-b3ff-caae1c47d9e1,0.911693,WETH,weETH,2025-08-13
3,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,tokemak,0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,300.0,315.474806,None,99.0,Odos,2025-08-13 04:26:45.587442+00:00,7,733428f7-7880-4c97-9477-14a86b25ab68,1.051583,WETH,osETH,2025-08-13
4,8453,0x4200000000000000000000000000000000000006,tokemak,0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22,0x4200000000000000000000000000000000000006,225.0,247.481143,None,99.0,Lifi,2025-08-13 04:23:51.539561+00:00,7,15f6112d-361a-4d84-8748-505dc9b6bb24,1.099916,WETH,cbETH,2025-08-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,odos,0xBC6DA0FE9aD5f3b0d58160288917AA56653660E9,0x865377367054516e17014CcdED1e7d814EDC9ce4,10000.0,9973.874956,None,99.0,Odos,2025-08-13 18:21:49.635979+00:00,20,6010c990-6f93-4e63-a5cf-30c623e2a56c,0.997387,DOLA,alUSD,2025-08-13
10496,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,odos,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0x865377367054516e17014CcdED1e7d814EDC9ce4,700000.0,832311.612059,None,99.0,Odos,2025-08-13 18:18:39.551871+00:00,20,43a87409-72bc-465a-90e8-c0076846150d,1.189017,DOLA,sUSDe,2025-08-13
10497,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,odos,0xae78736Cd615f374D3085123A210448E74Fc6393,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100.0,113.546429,None,99.0,Odos,2025-08-13 18:22:34.904841+00:00,20,9a85af1a-1f9d-4590-a65f-faa236b36ace,1.135464,WETH,rETH,2025-08-13
10498,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,tokemak,0xbf5495Efe5DB9ce00f80364C8B423567e58d2110,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,150.0,158.071997,None,99.0,Odos,2025-08-13 18:23:10.232322+00:00,20,28e855f9-a798-4e61-bf22-69bcfb57ec2b,1.053813,WETH,ezETH,2025-08-13


In [32]:
sell_token_choices = sorted(df["sell_token_symbol"].dropna().unique().tolist())
api_name_choices = sorted(df["api_name"].dropna().unique().tolist())
chain_id_choices = sorted(df["chain_id"].dropna().unique().tolist())
chain_id_choices

[1, 8453]

The issue is that this is not representitive

In [36]:
selected_api = "odos"
selected_chain = 1



for selected_symbol in sell_token_choices:
    this_token_df = df[
        (df["sell_token_symbol"] == selected_symbol)
        & (df["api_name"] == selected_api)
        & (df["chain_id"] == selected_chain)
    ].copy()

    if this_token_df.empty:
        continue

    # ensure numeric and aggregate to 1 point per x per batch
    this_token_df["scaled_amount_in"] = pd.to_numeric(this_token_df["scaled_amount_in"], errors="coerce")
    agg = (
        this_token_df
        .groupby(["quote_batch", "scaled_amount_in"], as_index=False)["effective_price"]
        .median()                         # or .min() if you want best price
        .sort_values(["quote_batch", "scaled_amount_in"])
    )

    px.line(
        agg,
        x="scaled_amount_in",
        y="effective_price",
        color="quote_batch",
        markers=True,
        title=f"{selected_symbol} {selected_api} Effective Price vs Amount In on Chain {selected_chain}",
    ).show()

# what kind of volititily is reasonable?


In [34]:
scrvUSD_df =df[(df['sell_token_symbol'] == 'scrvUSD') &
   (df['chain_id'] == 1) &
   (df['api_name'] == 'tokemak')
   ].copy()

scrvUSD_df['effective_price'].value_counts()

effective_price
1.059013    1
1.060460    1
1.059211    1
1.059217    1
1.059189    1
           ..
1.060558    1
1.059127    1
1.060856    1
1.059150    1
1.059224    1
Name: count, Length: 352, dtype: int64

In [35]:
px.line(.pivot_table(
    index='quote_batch',
    columns='scaled_amount_in',
    values='scaled_amount_out',
    aggfunc='mean'
).T)

# .groupby(['quote_batch', 'scaled_amount_in'])['scaled_amount_out'].mean().reset_index().sort_values('quote_batch')

SyntaxError: invalid syntax (239444765.py, line 1)

In [42]:
recent_df = df[df['quote_batch'] == 20].reset_index(drop=True)

scrvUSD_df =recent_df[(df['sell_token_symbol'] == 'scrvUSD') &
   (recent_df['chain_id'] == 1) &
   (recent_df['api_name'] == 'tokemak')
   ].reset_index()

scrvUSD_df

/var/folders/7y/zqhjt5cx12n1p79c5pfxszvm0000gn/T/ipykernel_34314/2925908884.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,index,chain_id,base_asset,api_name,sell_token_address,buy_token_address,scaled_amount_in,scaled_amount_out,pools_blacklist,percent_exclude_threshold,aggregator_name,datetime_received,quote_batch,id,effective_price,buy_token_symbol,sell_token_symbol,day
0,29,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,tokemak,0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,325.0,3.242798e+02,None,99.0,Odos,2025-08-13 18:27:27.348405+00:00,20,7813f6f7-b518-4869-aa71-89f07d32a007,0.997784,WETH,stETH,2025-08-13
1,86,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,tokemak,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0x865377367054516e17014CcdED1e7d814EDC9ce4,900000.0,1.072408e+06,None,99.0,4626,2025-08-13 18:15:39.117551+00:00,20,ac3bda8d-550a-447c-96eb-0b81407b20a8,1.191565,DOLA,sUSDe,2025-08-13
2,162,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,tokemak,0xBC6DA0FE9aD5f3b0d58160288917AA56653660E9,0x865377367054516e17014CcdED1e7d814EDC9ce4,400000.0,3.985602e+05,None,99.0,Odos,2025-08-13 18:28:08.230428+00:00,20,7e895a46-ba0f-433c-b4d9-03385a7a47c6,0.996401,DOLA,alUSD,2025-08-13
3,176,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,tokemak,0xae78736Cd615f374D3085123A210448E74Fc6393,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,50.0,5.678028e+01,None,99.0,Odos,2025-08-13 18:21:20.291240+00:00,20,d8efbd89-40f8-405d-889e-13bd930f34d8,1.135606,WETH,rETH,2025-08-13
4,186,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,tokemak,0xae78736Cd615f374D3085123A210448E74Fc6393,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,150.0,1.703088e+02,None,99.0,Odos,2025-08-13 18:20:34.991112+00:00,20,2ad36e14-f82e-4252-92f9-802c884e011a,1.135392,WETH,rETH,2025-08-13
5,217,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,tokemak,0xCd5fE23C85820F7B72D0926FC9b05b43E359b7ee,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,5.0,5.359576e+00,None,99.0,Odos,2025-08-13 18:24:22.632034+00:00,20,9790599f-4c17-4afe-994e-611a9f3c23af,1.071915,WETH,weETH,2025-08-13
6,274,1,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,tokemak,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,800000.0,7.998595e+05,None,99.0,Odos,2025-08-13 18:26:15.965614+00:00,20,ae25c812-fcc4-4c7c-9ee4-af5a01febedd,0.999824,USDC,GHO,2025-08-13
7,346,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,tokemak,0x57aB1E0003F623289CD798B1824Be09a793e4Bec,0x865377367054516e17014CcdED1e7d814EDC9ce4,800000.0,7.928691e+05,None,99.0,Odos,2025-08-13 18:20:46.979376+00:00,20,9b4d26f1-d2b7-437f-9b4e-65c326070dfd,0.991086,DOLA,reUSD,2025-08-13
8,440,1,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,tokemak,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,900000.0,8.998394e+05,None,99.0,Odos,2025-08-13 18:26:31.192415+00:00,20,f0e30063-9a11-4ba0-8e9a-3bc73b2d8e78,0.999822,USDC,GHO,2025-08-13
9,476,1,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,tokemak,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,400000.0,4.760179e+05,None,99.0,4626,2025-08-13 18:15:48.296262+00:00,20,ffce4178-6a47-45a1-b721-7a21a7cebcee,1.190045,USDC,sUSDe,2025-08-13


In [51]:
px.scatter(scrvUSD_df, x='scaled_amount_in', y='scaled_amount_out')